In [326]:
import numpy as np

class PIDController():

    AUTO_MODE = 1
    OFF_MODE = 0

    def __init__(self, pgain=1, itime=1, dtime=0, auto_max=100,
                 auto_min=0, beta=1, linearity=1, alpha=1, deadband=0,
                 sp_high=100, sp_low=0, gamma=0, man_request=0, mode=0):

        self.auto_min = auto_min
        self.auto_max = auto_max
        self.pgain = pgain
        self._itime = itime
        if self._itime:
            self.oneoveritime = 1 / self.itime
        else:
            self.oneoveritime = 0
        self.dtime = dtime
        self.g = gamma
        self.man_request = man_request

        self.bump = 0
        self.last_output = 0
        self.last_err = 0
        self.last_gerr = 0
        self.last_pv = 0
        self.b = beta
        self.l = linearity
        self.a = alpha
        self.deadband = deadband
        self._mode = mode
        
        self.sp_high = sp_high
        self.sp_low = sp_low
        
        self.Uk = self.Up = self.Ui = self.Ud = 0
        
    @property
    def mode(self):
        return self._mode
    
    def set_mode(self, m, pv, sp):
        if m not in (0,1,2):
            raise ValueError(m)
        if m == self._mode:
            return
        elif self._mode == 0 and m == 2:
            self.off_to_auto(pv, sp)
        elif self._mode == 1 and m == 2:
            self.man_to_auto(pv, sp, self.man_out)
        self._mode = m
        
    @property
    def itime(self):
        return self._itime

    @itime.setter
    def itime(self, v):
        self._itime = v
        if v:
            self.oneoveritime = 1 / v
        else:
            self.oneoveritime = 0

    def off_to_auto(self, pv, sp):
        """
        Calculate bump for off-to-auto transfer
        :param pv: current process temp to use for bumpless xfer calculation
        """
        self.man_to_auto(pv, sp, 0)

    def man_to_auto(self, pv, sp, op):
        err_for_pgain = (self.b*sp-pv)*(self.l+(1-self.l)*(abs(self.b*sp-pv))/100)
        uk0 = self.pgain * err_for_pgain
        self.Ui = op - uk0
        self.last_pv = pv
        self.last_err = 0
        self.last_ierr = 0
        self.last_gerr = self.g * sp - pv
        self.last_berr = self.b * sp - pv
        self.last_output = op
   
    set_bump = man_to_auto

    def reset(self):
        self.Ui = 0
        self.last_pv = 0
        self.last_err = 0
        self.last_ierr = 0
        self.last_berr = 0
    
    def step(self, pv, sp):
        
        if self._mode != 0:
            if self._mode == 1:
                return self.man_request
            elif self._mode == 2:
                return 0
            else:
                raise ValueError(self._mode)
        err = sp - pv
        berr = self.b * sp - pv
        gerr = self.g * sp - pv
        
        # Based on Labview's `PID Get Advanced Error (DBL).vi`
        if self.l == 1 or self.sp_high == self.sp_low:
            nl_int_factor = 1
        else:
            assert False
            sp_rng = self.sp_high - self.sp_low
            oml = 1 - self.l
            nl_int_factor = 1 / (1+err*err/ (sp_rng*sp_rng))
            err = err * (self.l + oml*abs(err)/sp_rng)
            berr = berr * (self.l + oml*abs(berr)/sp_rng)
            gerr = gerr * (self.l + oml*abs(gerr)/sp_rng)
        
        errs_avg = (err + self.last_err) / 2
        ierr = errs_avg * nl_int_factor * self.pgain * self.oneoveritime
        
        Up = self.pgain * berr
        Ui = self.Ui + ierr
        if 0 < self.a <= 1:
            Ud = (self.Ud - ((self.pgain / self.a) * (gerr - self.last_gerr))) * (self.dtime / (self.dtime + (1 / self.a)))
        else:
            Ud = -(self.last_gerr - gerr) * self.pgain * self.dtime 
        Uk = Up + Ui + Ud
        
        # Coercion & back calculation      
        if Uk > self.auto_max:
            Uk = self.auto_max
            Ui = Uk - Up - Ud
        elif Uk < self.auto_min:
            Uk = self.auto_min
            Ui = Uk - Up - Ud
            
        # XXX debugging
        self.Ui = Ui
        self.Uk = Uk
        self.Ud = Ud
        self.Up = Up
            
        # "Shift Register"
        self.last_output = Uk
        self.last_pv = pv
        self.last_err = err
        self.last_ierr = ierr
        self.last_gerr = gerr
        
        return Uk

    def __repr__(self):
        return "Output: %.2f Pgain: %.1f Itime: %.2f" % (self.last_output,
                                                                          self.pgain,
                                                                          self.itime)
    __str__ = __repr__
    

def assert_equal(a,b):
    assert a == b, (a, b)
def test_trap_integration1():
    data = [
        (0, 1, .5),
        (0, 2, 2),
        (0, 3, 4.5)
    ]
    p = PIDController(1,1)
    for pv, sp, op in data:
        p.Ui -= 1  # counteract effect of increasing set point on Up
        res_op = p.step(pv, sp)
        assert_equal(p.last_err, sp)
        assert_equal(p.Uk, op)
        assert_equal(res_op, op)
test_trap_integration1()

def check_Ui_backcalc(it, pg, Up, Ud, ierr, accum, val):
    Ui2 = it * (ierr + accum) * pg
    Uk2 = Up + Ud + Ui
    Uk2 = round(Uk2, 8)
    am = round(val, 8)
    assert Uk2 == am, (Up, Ud, Ui, Uk2, val)

In [327]:
def m2s(m):
    return m*60
def s2m(s):
    return s/60
def h2s(h):
    return m2s(h*60)

from types import FunctionType, MethodType
def printdir(o):
    for a in dir(o):
        if not a.startswith("__"):
            v = getattr(o, a)
            if not isinstance(v, (FunctionType, MethodType)):
                print(a, v)

from collections import deque
class DelayBuffer(deque):
    def __init__(self, delay=30, startvalue=0):
        delay = int(delay)
        self.delay = delay
        super().__init__(startvalue for _ in range(delay + 1))

    def cycle(self, hd):
        self[0] = hd
        self.rotate(1)
        return self[0]
    
class DelaySink():
    def __init__(self, delay, initial, pc_at_delay=0.95, dt=1):
        assert 0 < pc_at_delay <= 1, "don't be a retard"
        if delay:
            self.df = 1 - (1-pc_at_delay)**(dt/delay)
            self.sink = initial / self.df  - initial # fill the sink
        else:
            self.df = 0
            self.sink = 0
        self.dt = dt
        if self.sink < 0:
            self.sink = 0
        
    def delay(self, op):
        self.sink += op * self.dt
        dNdt = self.sink*self.df
        self.sink -= dNdt
        return dNdt

In [328]:
TOTAL_RVOLUME = {
    3: 4,
    15: 17,
    80: 90,
}

class DOProcess():
    """ DO Process
    Uses cascaded calculation model,
    First using HeadspaceProcess to estimate
    headspace gas concentrations, and using
    the result to calculate change in PV. 
    """
    # process gain constant units of  
    # % per delta% per hour 
    # dpv/dt = k*([O2]/0.2095 * 100 - DOPV)
    # where [02] is volume fraction, DOPV is in %
    default_k = 0.1306
    
    # consumption rate constant in units of % per hour
    # presumably, consumption rate is "Constant"
    # within approximation, as long as cellular conditions
    # don't transition between aerobic and anaerobic
    default_c = 0
    
    # volume fraction of CO2, N2, O2 in air
    # note the remaining 1% is trace gasses
    AIR_CNO = (0.0004, 0.7809, 0.2095)
    
    def __init__(self, main_gas=1, initial_pv=60, initial_cno=AIR_CNO, reactor_size=80, volume=55, delay=0):
        """
        :param g: gain in units of C/min/%
        :param k: decay rate in units of C/min/dT
        """
        self.tdelay = DelayBuffer(delay, initial_pv).cycle
        
        self.k = 0
        self.k_mult = 0
        self._k = 0
        self.c = 0
        self.dc = 0
        self.d2c = 0
        self.set_values(self.default_k, 1, self.default_c, 0, 0)
        
        self._reactor_size = reactor_size
        self._volume = volume
        self.main_gas = main_gas
    
        hs_volume = TOTAL_RVOLUME[reactor_size] - volume
        c, n, o = initial_cno
        self.hp = HeadspaceProcess(hs_volume, c, n, o)
        
    @property
    def volume(self): 
        return self._volume
    
    def set_values(self, k=None, k_mult=None, c=None, dc=None, d2c=None):
        if k is not None or k_mult is not None:
            if k is None:
                k = self.k * 3600
            else:
                self.k = k / 3600
            if k_mult is None:
                k_mult = self.k_mult
            else:
                self.k_mult = k_mult
            self._k = k_mult * k / 3600
    
        if c is not None:
            self.c = c / 3600
        if dc is not None:
            self.dc = dc / 3600 / 3600
        if d2c is not None:
            self.d2c = d2c / 3600 / 3600 / 3600
    
    @volume.setter
    def volume(self, v):
        self._volume = v
        self.hp.vol = TOTAL_RVOLUME[self._reactor_size] - v

    def step(self, pv, co2_req, n2_req, o2_req, air_req):
        self.hp.calc_gas(self.main_gas, co2_req, n2_req, o2_req, air_req)
        o2 = self.hp.o2A
        pvarg = (o2 * 100 / 0.2095 - pv)
        dpv = self._k * pvarg
        pv += dpv
        self.dc += self.d2c
        self.c += self.dc
        pv -= self.c
        if pv < 0:
            return 0
        return pv      
    

class HeadspaceProcess():
    def __init__(self, vol, co2A=0, n2A=0.78, o2A=0.21):
        """ 
        vol: volume headspace (L)
        main_gas: main gas flow rate (L/min)
        co2: CO2 request (%)
        n2: N2 request (%)
        
        Calculate headspace concentration. DT = 1 second. 
        """
        self.vol = vol
        self.co2A = co2A
        self.n2A = n2A
        self.o2A = o2A
        self.airA = 1 - (co2A+n2A+o2A)
        
    def step(self, main_gas, co2_req, n2_req, o2_req, air_req):
        self.calc_gas(main_gas, co2_req, n2_req, o2_req, air_req)
        return self.o2A
    
    def calc_gas(self, main_gas, co2_req, n2_req, o2_req, air_req):
        """ 
        Calculate new headspace gas concentration.
        Assumptions:
            - Mixing time is 0 seconds, i.e. gas mixture is always perfectly mixed
            - Gas lost from headspace is always equal to gas entering headspace
            - Concentration of individual gasses lost match headspace concentration of gasses. 
            - Mass transfer coefficient of Air <-> liquid k <<< Mg (no volume lost due to absorption to liquid)
            - Air O2: 20.95%, N2: 78.09%, CO2=0.04%, other = 1-(o2+n2+co2)
        """
        mg = main_gas / 60
        
        # volume of gas lost
        co2L = mg * self.co2A
        n2L = mg * self.n2A
        o2L = mg * self.o2A
        airL = mg * self.airA
        
        # volume of gas gained
        # airG is non-corrected volume of air added
        co2G = mg * co2_req
        n2G = mg * n2_req
        o2G = mg * o2_req
        airG = mg * air_req
        
        # air contribution to gases
        # airG is corrected to reflect
        # the volume of non-tracked gases
        # ie the 1% of trace elements or so
        co2G += 0.0004 * airG
        o2G += 0.2095 * airG
        n2G += 0.7809 * airG
        airG = .0092 * airG
        
        # new volume of each gas
        co2V = co2G - co2L + self.co2A * self.vol
        n2V = n2G - n2L + self.n2A * self.vol
        o2V = o2G - o2L + self.o2A * self.vol
        airV = airG - airL + self.airA * self.vol
        
        # finally, new percentages
        self.co2A = co2V / self.vol
        self.n2A = n2V / self.vol
        self.o2A = o2V / self.vol
        self.airA = airV / self.vol
        
        # Sanity checks
        # Errors should be on the order of 
        # floating point rounding error ~1e-14
        self._err_ = self.vol - sum((co2V, n2V, o2V, airV))
        self._err2_ = 1 - (self.co2A + self.n2A + self.o2A + self.airA)
        self._err3_ = (co2L+n2L+o2L+airL) - (co2G + n2G + o2G + airG)
        
    @staticmethod
    def actual_from_request(co2, n2, o2, air):
        total = co2 + n2 + o2 + air
        co2 /= total
        n2 /= total
        o2 /= total
        air /= total
        
        co2 += 0.0004 * air
        n2 += 0.7809 * air
        o2 += 0.2095 * air
        air = 1 - (co2+n2+o2)
        return co2, n2, o2, air
            

In [356]:
class GasController():
    def __init__(self, co2mfcmax, n2mfcmax, o2mfcmax, airmfcmax):
        self.cm = co2mfcmax
        self.nm = n2mfcmax
        self.om = o2mfcmax
        self.am = airmfcmax
   
    def request(self, main_gas, co2_req, n2_req, o2_req):

        co2max = self.cm / main_gas
        co2pc = co2max if co2_req > co2max else co2_req

        o2_req_max = 1 - co2pc
        if o2_req > o2_req_max: o2_req = o2_req_max
        o2max = self.om / main_gas
        o2pc = o2max if o2_req > o2max else o2_req
        
        n2_req_max = 1 - co2pc - o2pc
        if n2_req > n2_req_max: n2_req = n2_req_max
        n2max = self.nm / main_gas
        n2pc = n2max if n2_req > n2max else n2_req
        
        apc = 1 - co2pc - o2pc - n2pc
        return co2pc, n2pc, o2pc, apc

In [357]:
def _round(dig, *args):
    rv = []
    for a in args:
        rv.append(round(a, dig))
    return rv
def test_gas_controller():
    ctrl = GasController(1, 10, 2, 10)
    
    c,o,n = (0.8, 0.3, 1)
    rc, rn, ro, ra = _round(5, *ctrl.request(0.5, c,n,o))
    assert rc == 0.8, rc
    assert ro == 0.2, ro
    assert rn == 0, rn
    assert ra == 0, ra
    
    rc, rn, ro, ra = _round(5, *ctrl.request(10, c,n,o))
    assert rc == 0.1, rc
    assert ro == 0.2, ro
    assert rn == 0.7, rn
    assert ra == 0, ra
    
    c,o,n = (0.8, 0, 1)
    rc, rn, ro, ra = _round(5, *ctrl.request(10, c,n,o))
    assert rc == 0.1, rc
    assert ro == 0, ro
    assert rn == 0.9, rn
    assert ra == 0, ra
    
    c,o,n = (0.07, 0.1, 0.3)
    rc, rn, ro, ra = _round(5, *ctrl.request(10, c,n,o))
    assert rc == 0.07, rc
    assert ro == 0.1, ro
    assert rn == 0.3, rn
    assert ra == 0.53, ra
    
    for mg in (0.5, 1, 5, 10):
        for c in 0.1, 0.5, 0.8, 1:
            for o in 0.1, 0.5, 0.8, 1:
                for n in 0.1, 0.5, 0.8, 1:
                    args = (mg, c, n, o)
                    ec = round(min(1/mg, c), 5)
                    eo = round(min(2/mg, (1-ec), o),5)
                    en = round(min(1-ec-eo, n),5)
                    ea = round(1-ec-eo-en,5)
                    rc, rn, ro, ra = _round(5, *ctrl.request(*args))
                    assert rc == ec, (rc, ec, args)
                    assert ro == eo, (ro, eo, args, (rc, rn, ro, ra))
                    assert rn == en, (rn, en, args)
                    assert ra >= 0 and ra == ea, (ra, ea, args)
    
test_gas_controller()

In [358]:
import json

class OptionCategory():
    def __iter__(self):
        d = self.__class__.__dict__.copy()
        d.update(self.__dict__)
        ob = ((k, v) for k,v in d.items() if k[0] != "_" and k[-1]!="_")
        return iter(ob)
    
    def __repr__(self):
        buf = []
        for k, v in self:
            if isinstance(v, OptionCategory):
                v = "<More Options...>"
            buf.append("%s: %s" %(k,v))
        return "\n".join(buf)
    
    def __init__(self):
        self.__dict__.update(self.__class__.__dict__)
    
    def __setattr__(self, k, v):
        if k not in self.__dict__:
            raise AttributeError(k)
        object.__setattr__(self, k, v)
        
    def asdict(self):
        basedict = dict(iter(self))
        rv = {}
        for k in basedict:
            v = basedict[k]
            if isinstance(v, OptionCategory):
                v = v.asdict()
            rv[k] = v
        return rv
    
    def jsonify(self, **kw):
        kw['indent'] = kw.get('indent') or 4
        return json.dumps(self.asdict(), **kw)

class PIDOps(OptionCategory):
    p = 5
    i = 5
    d = 0
    amax = 100
    amin = 0
    alpha = 1
    beta = 1
    linearity = 1
    deadband = 0
    man_request = 0
    
class MFCOps(OptionCategory):
    co2_max = 1
    o2_max = 2
    n2_max = 10
    air_max = 10
    
class PlotOps(OptionCategory):
    xscale = 'auto'
    xmin = 0
    xmax = 72
    xscale_factor = 3600

class SimOps(OptionCategory):
    
    o2_pid = PIDOps()
    o2_pid.p = 5
    o2_pid.i = 5
    o2_pid.d = 0
    o2_pid.amax = 100
    o2_pid.amin = 0
    o2_pid.beta = 1
    o2_pid.linearity = 1
    o2_pid.alpha = 1
    
    n2_pid = PIDOps()
    n2_pid.p = 5
    n2_pid.i = 5
    n2_pid.d = 0
    n2_pid.amax = 0
    n2_pid.amin = 0
    n2_pid.beta = 1
    n2_pid.linearity = 1
    n2_pid.alpha = 1
    
    mfcs = MFCOps()
    mfcs.co2_max = 1
    mfcs.o2_max = 10
    mfcs.n2_max = 10
    mfcs.air_max = 10
    
    plots = PlotOps()
    plots.xscale = 'auto'
    plots.xmin = 0
    plots.xmax = 72
    plots.xscale_factor = 3600
    
    delay = 0
    end = 10000
    initial_actual_cno = DOProcess.AIR_CNO
    initial_request_cno = (0.07, 0, 0)
    initial_pv = 90
    set_point = 40
    set_point_deadband = 1
    k_mult = 1.1
    k = None
    c = None
    dc = 0
    d2c = 0
    mode = "m2a"
    main_gas = 1.0
    reactor_size = 80
    reactor_volume = reactor_size * 55/80


In [367]:

def do_sim(ops):
    o2_pid = PIDController(pgain=ops.o2_pid.p,              itime=m2s(ops.o2_pid.i),
                            dtime=m2s(ops.o2_pid.d),        auto_max=ops.o2_pid.amax,
                            auto_min=ops.o2_pid.amin,       
                            beta=ops.o2_pid.beta,
                            linearity=ops.o2_pid.linearity, alpha=ops.o2_pid.alpha,
                            deadband=ops.o2_pid.deadband)
    
    n2_pid = PIDController(pgain=ops.n2_pid.p,              itime=m2s(ops.n2_pid.i),
                            dtime=m2s(ops.n2_pid.d),        auto_max=ops.n2_pid.amax,
                            auto_min=ops.n2_pid.amin,       
                            beta=ops.n2_pid.beta,
                            linearity=ops.n2_pid.linearity, alpha=ops.n2_pid.alpha,
                            deadband=ops.n2_pid.deadband)
    
    co2_req, n2_req, o2_req = ops.initial_request_cno
    air_req = max(1-co2_req - n2_req - o2_req, 0)
    pv = ops.initial_pv
    sp = ops.set_point
    mode = ops.mode
    db = ops.set_point_deadband
    
    if mode == "o2a":
        n2_req = 0
        o2_req = 0
        mode = "m2a"  # fallthrough below
    
    if mode == 'm2a':
        o2_pid.man_to_auto(pv, sp-db, o2_req*100)
        n2_pid.man_to_auto(pv, sp+db, n2_req*100)
    else:
        o2_pid.auto_to_auto(pv, o2_req*100)
        n2_pid.auto_to_auto(pv, o2_req*100)
        
    delay = ops.delay
    mg = ops.main_gas
    co2a, n2a, o2a = ops.initial_actual_cno
    
    proc = DOProcess(mg, pv,     (co2a, n2a, o2a), 
                     ops.reactor_size, ops.reactor_volume, 
                     ops.delay)
    
    proc.set_values(ops.k*ops.k_mult, ops.c)
        
    ctrl = GasController(ops.mfcs.co2_max, ops.mfcs.n2_max, ops.mfcs.o2_max, ops.mfcs.air_max)
    
    t = 0
    end = ops.end
    
    data = [(t, pv, mg, co2_req, n2_req, o2_req, air_req, co2a, n2a, o2a)]
    plotting_pid = n2_pid
    data2 = [(plotting_pid.Uk, plotting_pid.Up, plotting_pid.Ui, plotting_pid.Ud)]
    while True:
        t += 1
        o2_req = o2_pid.step(pv, sp-db) / 100
        n2_req = n2_pid.step(pv, sp+db) / 100
        co2_req, n2_req, o2_req, air_req = ctrl.request(mg, co2_req, n2_req, o2_req)
        pv = proc.step(pv, co2_req, n2_req, o2_req, air_req)
        co2a = proc.hp.co2A
        n2a = proc.hp.n2A
        o2a = proc.hp.o2A
        data.append((t, pv, mg, co2_req, n2_req, o2_req, air_req, co2a, n2a, o2a))
        data2.append((plotting_pid.Uk, plotting_pid.Up, plotting_pid.Ui, plotting_pid.Ud))
        if t >= end:
            break
    return data, data2

In [368]:
ops = SimOps()
ops.delay = 0
ops.end = 10000
ops.initial_actual_cno = DOProcess.AIR_CNO
ops.initial_request_cno = (0.07, 0, 0)
ops.initial_pv = 100
ops.set_point = 90
ops.k = DOProcess.default_k
ops.c = 0
ops.mode = "o2a"
ops.main_gas = 1.0
ops.reactor_size = 80
ops.reactor_volume = 55

In [369]:
# O2 pid
ops.o2_pid.p = 2
ops.o2_pid.i = 10
ops.o2_pid.d = 0
ops.o2_pid.alpha = -1
ops.o2_pid.amax = 100
ops.o2_pid.amin = 0
ops.o2_pid.linearity = 1
ops.o2_pid.beta = 1

In [370]:
# N2 PID
ops.n2_pid.p = -5
ops.n2_pid.i = 5
ops.n2_pid.d = 0
ops.n2_pid.amax = 100
ops.n2_pid.amin = 0
ops.n2_pid.alpha = -1
ops.n2_pid.beta = 1
ops.n2_pid.linearity = 1

In [371]:
# O2 needs an extra padding space because the negative sign
# for N2 pgain doesn't count, i guesss
def fmt_float(f):
    if not f:
        return "0"
    f = str(f)
    if "." not in f:
        return f
    else:
        f = f.strip("0")
        if f[-1] == ".":
            f = f[:-1]
        if f[0] == ".":
            f = "0" + f
    return f
def get_text(ops):

    text = \
"""
O2: P=%3d  I=%5d  Initial PV: %3d  k: %.4f/hr (%sx Est.)
N2: P=%3d  I=%4d  Set Point:  %3d  c: %s%%/hr

 """ % (ops.o2_pid.p, ops.o2_pid.i,ops.initial_pv, ops.k*ops.k_mult, fmt_float(ops.k_mult),
        ops.n2_pid.p, ops.n2_pid.i, ops.set_point, fmt_float(ops.c))
    text = text.strip()
    return text

In [372]:
def run(ops):
    global data, data2, text
    data, data2 = do_sim(ops)
    text = get_text(ops)
    plot(ops)

In [373]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
%matplotlib

def plot(ops):    
    print("Parsing Data")
    global x, pv, mg, co2_req, n2_req, o2_req, air_req, co2a, n2a, o2a, uk, up, ui, ud
    x, pv, mg, co2_req, n2_req, o2_req, air_req, co2a, n2a, o2a = list(zip(*data))
    uk, up, ui, ud = list(zip(*data2))
    print("Closing Plot")
    plt.close()
    print("Plotting Data")

    step = 100
    xs = x[::step]
    xs = np.array(xs) / ops.plots.xscale_factor
    fig = plt.figure()
    ax1 = plt.subplot(311)
    ax2 = plt.subplot(312)
    ax3 = plt.subplot(313)

    pvs = np.array(pv[::step])
    ax1.plot(xs, pvs, "blue", ls="-", label="PV")
    ax1.axhline(y=sp, ls="--", color="black")
    # abs_err = np.abs(sp - pvs)
    # esq **= 2
    # esq = np.sqrt(esq)
    #ax1.plot(xs, abs_err, "red", ls="-", label="Err^2")

    #ax2.plot(xs, co2_req[::step], "purple", ls="-", label="CO2")
    ax2.plot(xs, n2_req[::step], "red", ls="-", label="N2")
    ax2.plot(xs, o2_req[::step], "green", ls="-", label="O2")
    ax2.plot(xs, air_req[::step], "cyan", ls="-", label="Air")
    fm1 = FuncFormatter(lambda y, _: "%.2f%%"%y)
    fm2 = FuncFormatter(lambda y, _: "%.2f%%"%(y*100))

    ax3.plot(xs, uk[::step], "blue", ls="-", label="Uk")
    ax3.plot(xs, up[::step], "red", ls="-", label="Up")
    ax3.plot(xs, ui[::step], "green", ls="-", label="Ui")
    ax3.plot(xs, ud[::step], "purple", ls="-", label="Ud")

    ax1.yaxis.set_major_formatter(fm1)
    ax2.yaxis.set_major_formatter(fm2)
    ax2.set_ylim((0, 1.1))
    
    m, ma = ax1.get_ylim()
    if np.min(pvs) - 1 < m:
        m -= 1
    if np.max(pvs) + 1 > ma:
        ma += 1
    ax1.set_ylim((m, ma))
    #ax1.set_ylim((sp-5, sp+5))


    for a in ax1, ax2, ax3: 
        b = a.get_position()
        a.set_position([b.x0, b.y0, b.width*0.9, b.height])
        a.legend(bbox_to_anchor=(0.99, 1.06), loc="upper left")
        a.grid()
        if ops.plots.xscale == 'man':
            a.set_xlim((ops.plots.xmin, ops.plots.xmax))
    
    fig.text(0.15, 0.95, get_text(ops), transform=ax1.transAxes, fontsize=12,
        verticalalignment='top')
    
    wm=plt.get_current_fig_manager()
    wm.window.attributes('-topmost', 1)
    wm.window.attributes('-topmost', 0)
    # h = wm.window.winfo_height()
    # w = wm.window.winfo_width()
    wm.window.geometry("%sx%s+%s+%s"%(700,720,50, 20))

Using matplotlib backend: TkAgg


In [374]:
seconds = 1
minutes = seconds * 60
hours = minutes * 60
days = hours * 24
ops.end = 10 * hours
sp = 80

ops.o2_pid.p = 2
ops.o2_pid.i = 40
ops.o2_pid.amax = 100
ops.o2_pid.deadband = 0
ops.o2_pid.d = 0
ops.o2_pid.alpha = -1

ops.n2_pid.p = -10
ops.n2_pid.i = 50
ops.n2_pid.amax = 100
ops.n2_pid.deadband = 0
ops.n2_pid.d = 0
ops.n2_pid.alpha = -1

ops.initial_pv = 20
ops.set_point = sp
ops.c = 0
ops.dc = 0
ops.d2c = 0

# k_mult = 1.25
ops.k_mult = 1.25
ops.k = DOProcess.default_k
n2 = 0.8 + 0.78*0.2
o2 = 0 + 0.2095 * 0.2
ops.initial_actual_cno = (0.0000, n2, o2)
ops.initial_request_cno = (0.00, n2, 0)
ops.mode = 'm2a'
ops.plots.xscale = 'auto'
ops.plots.xmin = 20
ops.plots.xmax = 30
ops.set_point_deadband = 1
ops.delay = 0

run(ops)

from matplotlib.ticker import MultipleLocator
l = MultipleLocator(1)
for a in plt.gcf().axes:
    a.xaxis.set_major_locator(l)

Parsing Data
Closing Plot
Plotting Data


In [18]:
plot(ops)

Parsing Data
Closing Plot
Plotting Data


In [19]:
# Maximum value of c that O2 controller can sustain for a given k
ops.k*ops.k_mult*(100/20.95)*100

77.92362768496422

In [20]:
# manually constructed list of [(o2p, o2i),(n2p, n2i)]
maybe_good_params = [
    [
        (10, 10),
        (-10, 100)
    ],
    [
        (2, 40),
        (-10, 50)
    ]
]

In [345]:
class RingBuffer():
    """ Numpy-based ring buffer """

    def __init__(self, maxsize, dtype=np.float32):
        if not maxsize or maxsize < 0:
            raise ValueError("%s requires max size argument" % self.__class__.__name__)
        self._queue = np.zeros(maxsize, dtype)
        self._dtype = dtype
        self._maxsize = maxsize
        self._end = 0
        self._sz = 0

    def put(self, d):
        self._queue[self._end] = d
        self._end += 1
        if self._end == self._maxsize:
            self._end = 0
        if self._sz < self._maxsize:
            self._sz += 1

    def put_list(self, lst):
        """
        :param lst: list to extend data from
        :type lst: list | tuple | np.ndarray
        """
        slen = len(lst)
        if slen > self._maxsize:
            self._queue = np.array(lst, dtype=self._dtype)[slen-self._maxsize:]
            self._sz = self._maxsize
            self._end = 0
            return
        if slen + self._end <= self._maxsize:
            start = self._end
            end = slen + self._end
            self._queue[start:end] = lst
            self._end = end
        else:
            # add slice in two steps
            first_step = self._maxsize - self._end
            self._queue[self._end: self._maxsize] = lst[:first_step]
            second_step = slen - first_step
            self._queue[:second_step] = lst[first_step:]
            self._end = second_step

        if self._sz < self._maxsize:
            self._sz += slen
        if self._end == self._maxsize:
            self._end = 0

    def extend(self, it):
        try:
            it.__len__  # list, tuple, nparray
        except AttributeError:
            it = tuple(it)
        self.put_list(it)

    def get(self):
        """
        Get method. Returns the entire queue but does NOT
        drain the queue.
        """
        if self._sz < self._maxsize:
            return self._queue[: self._end]
        else:
            return np.roll(self._queue, -self._end)

    def __len__(self):
        return self._sz

    def clear(self):
        self._end = 0
        self._sz = 0

In [346]:
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg
from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure
from matplotlib.ticker import MultipleLocator
import tkinter as tk


class Data():
    def __init__(self, pts):
        self.values = RingBuffer(pts)
        self.pending = []
        self.put = self.pending.append
        
    def __len__(self):
        return len(self.values)
        
    def push(self, n):
        self.values.extend(self.pending[:n])
        self.pending = self.pending[n:]
        self.put = self.pending.append
        
    def get(self):
        return self.values.get()

    def clear(self):
        self.values.clear()
        self.pending.clear()
        
    def resize(self, new_sz):
        nv = RingBuffer(new_sz)
        self.values.extend(self.pending)
        self.pending = []
        self.put = self.pending.append
        nv.extend(self.values.get())
        self.values = nv

class SeriesData(Data):
    def __init__(self, name, ax, xdata, pts, **line_kw):
        self.name = name
        super().__init__(pts)
        self.x = xdata
        self.line, = ax.plot((), (), **line_kw)
        
    def update_line(self):
        self.line.set_data(self.x.get(), self.values.get())
        
    def hide(self):
        self.line.set_visible(False)
        
    def show(self):
        self.line.set_visible(False)
        
    def set_visible(self, visible):
        self.line.set_visible(visible)
        
    def get_visible(self):
        return self.line.get_visible()
    
    def toggle_visible(self):
        self.set_visible(not self.get_visible())
        
    
# Plot classes contain custom configuration code
# so SimWindow doesn't need to know how to set them up

class Plot():
    def __init__(self, ax, xdata, pts, yformatter=None, *series_ops):
        self.ax = ax
        self.x = xdata
        self.series = []
        for ops in series_ops:
            name, kw = ops
            # default
            kw['ls'] = kw.get('ls') or "-"
            s = SeriesData(name, ax, xdata, pts, **kw)
            self.series.append(s)
            setattr(self, name, s)
        b = ax.get_position()
        ax.set_position([b.x0, b.y0, b.width*0.9, b.height])
        ax.legend(bbox_to_anchor=(0.99, 1.06), loc="upper left")
        self.leg_line_map = {}
        for lline, s in zip(ax.legend_.get_lines(), self.series):
            self.leg_line_map[lline] = s
        ax.grid()
        if yformatter:
            ax.yaxis.set_major_formatter(yformatter)
        self._hline = None
        
    def get_leg_map(self):
        return self.leg_line_map.copy()
            
    def put(self, *values):
        if len(values) != len(self.series):
            raise ValueError("Invalid argument to put: got %d values (expected %d)" \
                            % (len(values), len(self.series)))
        for s, v in zip(self.series, values):
            s.put(v)
            
    def push(self, n):
        for s in self.series:
            s.push(n)
            
    def update(self):
        for s in self.series:
            s.update_line()
        self.rescale()
        
    def rescale(self):
        self.ax.relim(True)
        self.ax.autoscale_view(True,True,True)
        
    def axhline(self, y, **kw):
        kw['ls'] = kw.get('ls') or "--"
        kw['color'] = kw.get('color') or 'black'
        if self._hline:
            self._hline.remove()
        self._hline = self.ax.axhline(y=y, **kw)
        
    def set_tick_interval(self, n):
        l = MultipleLocator(n)
        self.ax.xaxis.set_major_locator(l)
        
    def clear(self):
        for s in self.series:
            s.clear()
            

class PVPlot(Plot):
    """ Plot for PV """
    def __init__(self, ax, xdata, pts):
        series = [
            ("pv", dict(color="blue", ls="-", label="PV")),
            ("c", dict(color="green", ls="-", label="c"))
        ]
        fm = FuncFormatter(lambda y, _: "%.2f%%"%y)
        super().__init__(ax, xdata, pts, fm, *series)
        self.c.line.set_visible(False)
        
    def rescale(self):
        self.ax.relim(True)
        self.ax.autoscale_view(True,True,True)
        lower, upper = self.ax.get_ybound()
        padding = 5
        self.ax.set_ylim(lower - padding, upper + padding, True, None)
    
class GasesPlot(Plot):
    def __init__(self, ax, xdata, pts):
        
        series = [
            ("co2", dict(color="purple", ls="-", label="CO2")),
            ("n2", dict(color="red", ls="-", label="N2")),
            ("o2", dict(color="green", ls="-", label="O2")),
            ("air", dict(color="cyan", ls="-", label="Air")),
        ]
        fm = FuncFormatter(lambda y, _: "%.2f%%"%(y*100))
        super().__init__(ax, xdata, pts, fm, *series)
        self.ax.set_ylim((-0.05, 1.1))
        
    def rescale(self):
        self.ax.relim(True)
        self.ax.autoscale_view(True,True,False)
        
        
class PIDPlot(Plot):
    def __init__(self, ax, xdata, pts):
        series = [
            ("uk", dict(color="blue", ls="-", label="Uk")),
            ("up", dict(color="red", ls="-", label="Up")),
            ("ui", dict(color="green", ls="-", label="Ui")),
            ("ud", dict(color="purple", ls="-", label="Ud")),
        ]
        
        super().__init__(ax, xdata, pts, None, *series)

In [347]:
class SimConfig(OptionCategory):
    simops = ops or SimOps()
    xwindow_hrs = 20
    pause = False
    update_interval = 100
    time_factor = 200
    time_unit = 3600
    max_iters = hours * 24 * 7  # max number of iters to simulate in one graph update interval
    x_tick_interval = 1         # spacing between ticks (in hours)
    
    fig_width  = 7  # inches
    fig_height = 9 # inches
cfg = SimConfig()

In [348]:
class EventDispatcher():
    def __init__(self):
        self.handlers = {}
        
    def create_event(self, ev):
        self.handlers[ev] = self.handlers.get(ev, [])
        
    def register(self, ev, handler):
        if ev not in self.handlers:
            self.create_event(ev)
        self.handlers[ev].append(handler)
        
    def unregister(self, ev, handler):
        self.handlers[ev].remove(handler)
        
    def fire(self, ev, *args):
        for h in self.handlers.get(ev, ()):
            h(ev, *args)
            
    def unregister_all(self, ev):
        self.handlers[ev] = []
        
    def create_callback(self, ev, *args):
        def cb():
            self.fire(ev, *args)
        return cb
            

In [349]:
def _setpid(ob, state, name, value):
    pid = state[ob]
    if name == "pgain":
        pid.pgain = value
    elif name == "itime":
        pid.itime = m2s(value)
    elif name == "dtime":
        pid.dtime = m2s(value)
    elif name == "beta":
        pid.b = value
    elif name == "mode":
        sp = state['sp']
        if ob[:2] == "n2":
            sp += state['db']
        elif ob[:2] == 'o2':
            sp -= state['db']
        else:
            print("BAD PID")
        pid.set_mode(value, state['pv'], sp)
    else:
        print("WARNING: Invalid attribute for %r: %r" % (ob, name))

def _update_value(ops, state, proc, ob, name, value):
    if ob == "process":
        proc.set_values(**{name:value})
        return
    elif ob in ("o2_pid", "n2_pid"):
        _setpid(ob, state, name, value)
        return
    print("WARNING: Invalid attribute or object: %r %r" % (ob, name))


def do_sim_coroutine(self, xq, pvq, cq,
                     co2q, n2q, o2q, aq, 
                     nukq, nupq, nuiq, nudq,
                     oukq, oupq, ouiq, oudq):
    
    """ This is the longest function in the history of man. 
    Its long becaues there are a *lot* of values to use and unpack for the inner PID loop,
    including code needed to synchronize with UI updates.
    Yay. 
    """
    
    ops = self.cfg.simops
    o2_pid = PIDController(pgain=ops.o2_pid.p,              itime=m2s(ops.o2_pid.i),
                            dtime=m2s(ops.o2_pid.d),        auto_max=ops.o2_pid.amax,
                            auto_min=ops.o2_pid.amin,       
                            beta=ops.o2_pid.beta,
                            linearity=ops.o2_pid.linearity, alpha=ops.o2_pid.alpha,
                            deadband=ops.o2_pid.deadband)
    
    n2_pid = PIDController(pgain=ops.n2_pid.p,              itime=m2s(ops.n2_pid.i),
                            dtime=m2s(ops.n2_pid.d),        auto_max=ops.n2_pid.amax,
                            auto_min=ops.n2_pid.amin,      
                            beta=ops.n2_pid.beta,
                            linearity=ops.n2_pid.linearity, alpha=ops.n2_pid.alpha,
                            deadband=ops.n2_pid.deadband)
    
    co2_req, n2_req, o2_req = ops.initial_request_cno
    air_req = max(1-co2_req - n2_req - o2_req, 0)
    pv = ops.initial_pv
    sp = ops.set_point
    mode = ops.mode
    db = ops.set_point_deadband
    
    if mode == "o2a":
        n2_req = 0
        o2_req = 0
        mode = "m2a"  # fallthrough below
    
    if mode == 'm2a':
        o2_pid.man_to_auto(pv, sp-db, o2_req*100)
        n2_pid.man_to_auto(pv, sp+db, n2_req*100)
    else:
        o2_pid.auto_to_auto(pv, o2_req*100)
        n2_pid.auto_to_auto(pv, o2_req*100)
        
    delay = ops.delay
    mg = ops.main_gas
    co2a, n2a, o2a = ops.initial_actual_cno
    
    proc = DOProcess(mg, pv,     (co2a, n2a, o2a), 
                     ops.reactor_size, ops.reactor_volume, 
                     ops.delay)
    
    proc.set_values(ops.k, ops.k_mult, ops.c)
        
    ctrl = GasController(ops.mfcs.co2_max, ops.mfcs.n2_max, ops.mfcs.o2_max, ops.mfcs.air_max)
    
    t = 0
    time_unit = cfg.time_unit
    mi = cfg.max_iters
    msg = None
    state = self._current_state
    while True:

        cmd, arg = yield msg
        msg = None

        if cmd == "SIM_ITERS":
            
            # Note that this code allows the loop to execute even 
            # if iters == 0 (but no iterations are actually simulated).
            # This has the convenient side effect of allowing state status
            # updates to occur (for example, due to UI changes) even
            # when the simulation is paused (time factor = 0). 
            
            iters = arg
            if iters < 0:
                continue
            elif iters > mi:
                iters = mi

            # One approximation made in this simulation
            # is that no values, including oxygen consumption
            # (c), change during the inner iteration period 
            # outside of modification by the process or PID
            # classes themselves. 

            while iters > 0:

                t += 1
                iters -= 1
                o2_req = o2_pid.step(pv, sp-db) / 100
                n2_req = n2_pid.step(pv, sp+db) / 100
                co2_req, n2_req, o2_req, air_req = ctrl.request(mg, co2_req, n2_req, o2_req)
                pv = proc.step(pv, co2_req, n2_req, o2_req, air_req)

                xq.put(t/time_unit)
                pvq.put(pv)
                cq.put(proc.c * 3600)

                co2q.put(co2_req)
                n2q.put(n2_req)
                o2q.put(o2_req)
                aq.put(air_req)

                nukq.put(n2_pid.Uk)
                nupq.put(n2_pid.Up)
                nuiq.put(n2_pid.Ui)
                nudq.put(n2_pid.Ud)

                oukq.put(o2_pid.Uk)
                oupq.put(o2_pid.Up)
                ouiq.put(o2_pid.Ui)
                oudq.put(o2_pid.Ud)
                
            state['c'] = proc.c * 3600
            state['dc'] = proc.dc * 3600 * 3600
            state['d2c'] = proc.d2c * 3600 * 3600 * 3600
            state['k'] = proc.k
            state['sp'] = sp
            state['db'] = db
            state['pv'] = pv
            state['t'] = t
            state['co2_req'] = co2_req
            state['n2_req'] = n2_req
            state['o2_req'] = o2_req
            state['air_req'] = air_req
            state['o2_pid'] = o2_pid
            state['n2_pid'] = n2_pid

        elif cmd == "SET_TIME":
            t = arg
        elif cmd == "UPDATE_VALUE":
            ob, name, value = arg
            if ob is None:
                if name == "sp":
                    sp = value
                elif name == "pv":
                    pv = value
                else:
                    print("WARNING: Unrecognized name: %r" % name)
            else:
                rsp = _update_value(ops, state, proc, ob, name, value)
        else:
            raise ValueError(cmd)


In [350]:
class DOSimWindow():
    """ Sim Window handles common MPL functions & data storage 
    as well as non-MPL tkinter configuration 
    """
    def __init__(self, frame, sim_config, events):
        
        # Unpack config items
        self.cfg = sim_config
        self.update_interval = cfg.update_interval
        self.xwindow = cfg.xwindow_hrs
        
        # MPL objects
        self.fig = Figure(figsize=(self.cfg.fig_width, self.cfg.fig_height))
        ax1 = self.fig.add_subplot(411)
        ax2 = self.fig.add_subplot(412)
        ax3 = self.fig.add_subplot(413)
        ax4 = self.fig.add_subplot(414)
        
        self.xdata = Data(self.xwindow*3600)
        self.pv_plot = PVPlot(ax1, self.xdata, self.xwindow*3600)
        self.gases_plot = GasesPlot(ax2, self.xdata, self.xwindow*3600)
        self.n2_pid_plot = PIDPlot(ax3, self.xdata, self.xwindow*3600)
        self.o2_pid_plot = PIDPlot(ax4, self.xdata, self.xwindow*3600)
        
        x_spacing = self.cfg.time_unit / (self.cfg.x_tick_interval * 3600)
        self.pv_plot.set_tick_interval(x_spacing)
        self.gases_plot.set_tick_interval(x_spacing)
        self.n2_pid_plot.set_tick_interval(x_spacing)
        self.o2_pid_plot.set_tick_interval(x_spacing)

        # misc
        self.pv_plot.axhline(self.cfg.simops.set_point)
        
        # Tkinter setup
        self.root = frame
        
        self.fig_frame = tk.Frame(self.root)
        
        # a tk.DrawingArea
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.fig_frame)
        self.canvas.show()
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

        self.toolbar = NavigationToolbar2TkAgg(self.canvas, self.root)
        self.toolbar.update()
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
        
        self._running = False
        self._monitor_id = None
        self._monitor_interval = 500
        
        # coroutine support
        self.do_coro = None
        self._current_state = {}
        
        # Periodic update support
        self.updating = True
        
        # Event handling for callbacks
        # All event handling works by modifying
        # the config object directly. 
        # Each iteration set of the calculation loop
        # refreshes all values from the config object.
        # Long term, this should be converted to a command
        # queue. 
        
        self.events = events
        def on_setpoint_changed(ev, value, *args):
            self.trigger_update_value(None, "sp", value)
            self.pv_plot.axhline(value)
            print("Value Changed: sp %s" % value)
        self.events.register("SP_CHANGED", on_setpoint_changed)
        
        def on_pv_changed(ev, value, *args):
            self.trigger_update_value(None, "pv", value)
            print("Value Changed: pv %s" % value)
        self.events.register("PV_CHANGED", on_pv_changed)
        
        def on_pid_changed(ev, value, *args):
            ev = ev.lower()
            ctrl, param, _ = ev.split("_")
            ctrl += "_pid"
            print("Value Changed: %s.%s: %.2f" % (ctrl, param, value))
            self.trigger_update_value(ctrl, param, value)
            
        for c in "N2", "O2":
            for p in "PGAIN", "ITIME", "DTIME", "BETA", "MODE":
                ev = "%s_%s_CHANGED" % (c,p)
                self.events.register(ev, on_pid_changed)
        
        def on_process_gain_changed(ev, value, *args):
            c = ev.split("_")[0].lower()
            if c == "kmult":
                c = "k_mult"
            self.trigger_update_value("process", c, value)
            print("Changed value: %s to %s" % (c, value))
            
        for c in "K", "KMULT", "C", "DC", "D2C":
            ev = c + "_CHANGED"
            self.events.register(ev, on_process_gain_changed)
            
        def on_timefactor_changed(ev, value, *args):
            self.cfg.time_factor = value
            print(ev, ":", value)
        self.events.register("TIMEFACTOR_CHANGED", on_timefactor_changed)
        
        def on_clear_all_data(ev, *args):
            self.xdata.clear()
            for p in self.iter_plots():
                p.clear()
            self.do_coro.send(("SET_TIME", 0))
        self.events.register("CLEAR_ALL_DATA", on_clear_all_data)
        
        def on_advance_sim(ev, value, *args):
            if not value: return
            try:
                value = int(value)
            except ValueError:
                print("Invalid value: %r" % value)
            print("Advancing simulation %d seconds..." % value)
            self._run_sim(value)
        self.events.register("ADVANCE_SIM", on_advance_sim)
        
        def on_legend_clicked(event):
            ll = event.artist
            series = self.leg_map[ll]
            series.toggle_visible()
            
        def on_xwindow_changed(ev, value, *args):
            nv = int(value * 3600)
            for p in self.iter_plots():
                for s in p.series:
                    if len(s) != nv:
                        s.resize(nv)
            if len(self.xdata) != nv:
                self.xdata.resize(nv)
        self.events.register("XWINDOW_CHANGED", on_xwindow_changed)
            
        # legend picker support
        self.leg_map = {}
        for p in self.iter_plots():
            m = p.get_leg_map()
            for leg, s in m.items():
                leg.set_picker(5)
                self.leg_map[leg] = s
        self.fig.canvas.mpl_connect("pick_event", on_legend_clicked)

        
    def trigger_update_value(self, ob, name, value):
        cmd = "UPDATE_VALUE"
        args = (ob, name, value)
        self.do_coro.send((cmd, args))
        
    def iter_plots(self):
        return (self.pv_plot, self.gases_plot, self.n2_pid_plot, self.o2_pid_plot)
        
    def pack(self, side=tk.TOP, fill=tk.BOTH, expand=1):
        self.fig_frame.pack(side=side, fill=fill, expand=expand)
        
    def start(self):
        self._updating = True
        self.begin_sim()
        self.schedule_update()
        
    def stop(self):
        self._updating = False
        
    def resume(self):
        if not self.do_coro:
            self.start()
        else:
            self.schedule_update()
        
    def begin_sim(self):
        self.do_coro = do_sim_coroutine(self,
                         self.xdata, self.pv_plot.pv, self.pv_plot.c, self.gases_plot.co2,
                         self.gases_plot.n2, self.gases_plot.o2, self.gases_plot.air,
                         self.n2_pid_plot.uk, self.n2_pid_plot.up, self.n2_pid_plot.ui, self.n2_pid_plot.ud,
                         self.o2_pid_plot.uk, self.o2_pid_plot.up, self.o2_pid_plot.ui, self.o2_pid_plot.ud)
        next(self.do_coro)
        
    def update(self):
        n = self.cfg.time_factor
        n = int(n)
        self._run_sim(n)
        self.events.fire("PROCESS_STATE_UPDATE", self._current_state.copy())
        
    def _run_sim(self, n):
        self.do_coro.send(("SIM_ITERS", n))
        
        self.xdata.push(n)
        self.pv_plot.push(n)
        self.gases_plot.push(n)
        self.o2_pid_plot.push(n)
        self.n2_pid_plot.push(n)
        
        self.pv_plot.update()
        self.gases_plot.update()
        self.n2_pid_plot.update()
        self.o2_pid_plot.update()
        
        self.fig.canvas.draw()
        
    def schedule_update(self):
        def update(self=self):
            if self.updating:
                self.update()
                self.root.after(self.update_interval, update)
        self.root.after(self.update_interval, update)    
    


In [351]:
class TkQuitHack(tk.Tk):
    def _hacky_monitor_running(self):
        # hack to allow closing tkinter window by 
        # "X" button in this interactive prompt
        # unsure if this is an ipython/mpl artifact or not
        # but it works
        def monitor():
            if not self._hacky_running:
                for cb in self._hacky_on_destroyed:
                    cb()
                self.quit()
                self.destroy()
                self._hacky_destroyed = True
                
            else:
                self.after(self._hacky_monitor_interval, monitor)
        self.after(self._hacky_monitor_interval, monitor)
    def __init__(self, *args, **kw):
        super().__init__(*args, **kw)
        self._hacky_running = True
        self._hacky_monitor_interval = 200
        self._hacky_monitor_running()
        self.protocol("WM_DELETE_WINDOW", self.hacky_destroy)
        self._hacky_destroyed = False
        self._hacky_on_destroyed = []
        
    def hacky_destroy(self):
        self._hacky_running = False
        
    def register_destroy_callback(self, cb):
        self._hacky_on_destroyed.append(cb)
        
    def is_destroyed(self):
        return self._hacky_destroyed

In [352]:
import tkinter as tk, tkinter.ttk as ttk
class LabeledEntry(tk.Frame):
    def __init__(self, master, text, def_value, events, value_changed_event, pos="top", *args, **kw):
        super().__init__(master, *args, **kw)
        self.label = tk.Label(self, text=text)
        self.entry = tk.Entry(self)
        
        if pos.lower() == "top":
            r1, c1 = 0, 0
            r2, c2 = 1, 0
        elif pos.lower() == "left":
            r1, c1 = 0, 0
            r2, c2 = 0, 1
        elif pos.lower() == "right":
            r1, c1 = 0, 1
            r2, c2 = 0, 0
        else:
            raise ValueError("Bad Position: %r" % pos)
            
        self.label.grid(row=r1, column=c1, sticky=(tk.E, tk.W))
        self.entry.grid(row=r2, column=c2, sticky=(tk.E, tk.W))
            
        if def_value is None:
            def_value = ""
        self.value = def_value
        self.set(def_value)
        self._trigger_mode = "on_change"
            
        self.entry.bind("<Return>", self.maybe_value_changed)
        self.entry.bind("<FocusOut>", self.maybe_value_changed)
        self.events = events
        self.value_changed_event = value_changed_event
        
    def label_width(self, value):
        self.label.config(width=value)
        
    def entry_width(self, value):
        self.entry.config(width=value)
        
    def maybe_value_changed(self, _):
        value = self.get()
            
        try:
            v = float(value or 0)
        except ValueError:
            print("Could not coerce string to float: %r" % value)
            return
        
        if v == float(self.value) and self._trigger_mode == "on_change":
            return
        
        self.value = value
        self.trigger_value_changed(v)
        
    def trigger_value_changed(self, v):
            self.events.fire(self.value_changed_event, v)
        
    def get(self):
        return self.entry.get()
    
    def set(self, value):
        value = str(value)
        if not value:
            value = "0"
        self.entry.delete(0, tk.END)
        self.entry.insert(0, value)
        self.value = value
        
    def set_trigger_mode(self, mode):
        if mode not in ("always", "on_change"):
            raise ValueError("Unrecognized mode: %r" % mode)
        self._trigger_mode = mode
        
        
class EntryButton(tk.Frame):
    def __init__(self, master, text, events, event):
        super().__init__(master)
        self.entry = tk.Entry(self)
        self.entry.insert(0, 3600)
        self.button = ttk.Button(self, text=text, command=self.fire)
        self.entry.bind("<Return>", self.fire)
        self.events = events
        self.event = event
        
        self.entry.grid(row=0, column=1)
        self.button.grid(row=1, column=1)
        
    def fire(self):
        self.events.fire(self.event, self.entry.get())
    

# Patched tkinter settit. For some reason, setting the internal
# Variable isn't properly being used to update the OptionMenu
# widget text. 
      
# Also, this makes more sense as a function constructor
# than a class (imo)
    
def _setit(menu, var, value, cb=None):
    def func(*args):
        var.set(value)
        menu.config(text=value)
        if cb is not None:
            cb(value, *args)
    return func
        
# Patched tkinter optionmenu to work properly. The only changes
# are changing the default "textvariable" to "text" in kw to super,
# and re-running the code to bind to the patched _settit class. 

class PatchedOptionMenu(tk.Menubutton):
    """OptionMenu which allows the user to select a value from a menu."""
    def __init__(self, master, variable, value, *values, **kwargs):
        """Construct an optionmenu widget with the parent MASTER, with
        the resource textvariable set to VARIABLE, the initially selected
        value VALUE, the other menu values VALUES and an additional
        keyword argument command."""
        kw = {"borderwidth": 2, "text": value,
              "indicatoron": 1, "relief": tk.RAISED, "anchor": "c",
              "highlightthickness": 2}
        tk.Widget.__init__(self, master, "menubutton", kw)
        self.widgetName = 'tk_optionMenu'
        menu = self.__menu = tk.Menu(self, name="menu", tearoff=0)
        self.menuname = menu._w
        # 'command' is the only supported keyword
        callback = kwargs.get('command')
        if 'command' in kwargs:
            del kwargs['command']
        if kwargs:
            raise TclError('unknown option -'+kwargs.keys()[0])
        menu.add_command(label=value,
                 command=_setit(self, variable, value, callback))
        for v in values:
            menu.add_command(label=v,
                     command=_setit(self, variable, v, callback))
        self["menu"] = menu

    def __getitem__(self, name):
        if name == 'menu':
            return self.__menu
        return tk.Widget.__getitem__(self, name)

    def destroy(self):
        """Destroy this widget and the associated menu."""
        tk.Menubutton.destroy(self)
        self.__menu = None
        
class MenuFrame(tk.LabelFrame):
    def __init__(self, master, text, events, event, entries):
        super().__init__(master, text=text)
        self.entries = entries
        self.events = events
        self.event = event
        
        self.entries = []
        entry_names = []
        
        for i, (var, val, ev) in enumerate(entries, 1):
            w = LabeledEntry(self, var, val, events, ev)
            w.grid(row=i, column=0)
            self.entries.append(w)
            entry_names.append(var)
        
        self.tkvar = tk.StringVar(self)
        self.tkvar.set(entry_names[0])
        self.menu = OptionMenu(self, self.tkvar, *entry_names)
        self.menu.grid(column=0, row=0)
        self.events.register(self.event, self.on_menu_changed)
            
    def on_menu_changed(self, _):
        self.events.fire(self.event, self.tkvar.get())
        
            
class LabelFrameWithLabelEntries(tk.LabelFrame):
    def __init__(self, master, text, events, entries=()):
        super().__init__(master, text=text)
        self.events = events
        self.entries = []
        for text, val, ev in entries:
            self.add_entry(text, val, ev)
            
    def set_trigger_mode(self, mode):
        for e in self.entries:
            e.set_trigger_mode(mode)
            
    def add_entry(self, text, val, ev):
        e = LabeledEntry(self, text, val, self.events, ev, pos="left")
        e.label_width(10)
        e.entry_width(8)
        e.grid(row=len(self.entries), column=0)
        self.entries.append(e)
            
class LabelDisplay(tk.Frame):
    def __init__(self, master, events, event, text, value="", w1=None, w2=None):
        super().__init__(master)
        self.label = tk.Label(self, text=text, anchor=tk.E)
        self.display = tk.Label(self, text=value, anchor=tk.E)
        if w1:
            self.label.config(width=w1)
        if w2:
            self.display.config(width=w2)
        
        self.label.grid(row=0, column=0, sticky=(tk.W, tk.E))
        self.display.grid(row=0, column=1, sticky=(tk.W, tk.E))
        self.events = events
        self.events.register(event, self.on_value_update)
        
    def on_value_update(self, ev, value, *args):
        self.set(value)
        
    def set(self, value):
        value = str(value)
        self.display['text'] = value
    
    def get(self):
        return self.display['text']
        
class LabelDisplayFrame(tk.LabelFrame):
    def __init__(self, master, text, events):
        super().__init__(master, text=text)
        self.labels = {}
        self.events = events
        
    def add_label(self, id, event, text, value="", w1=None, w2=None):
        label = LabelDisplay(self, self.events, event, text, value, w1, w2)
        label.grid(row=len(self.labels), column=0, sticky=(tk.E, tk.W, tk.N, tk.S))
        self.labels[id] = label
        
    def add_labels(self, *args):
        for a in args:
            self.add_label(*args)
        
class Combobox(ttk.Combobox):
    def __init__(self, master, **kw):
        super().__init__(master, **kw)
        vcmd = self.register(self.nomodify,)
        self.config(validate="key", validatecommand=vcmd)
        self.bind("<Return>", lambda _: self.event_generate("<1>"))
        self.bind("<Button-1>", lambda _: self.event_generate("<Down>", when='head'))
        self.current(0)
        
    def nomodify(self):
        return False
        
class MenuLabel(tk.Frame):
    def __init__(self, master, text, events, event, values, w1=10, w2=6, map=None, **kw):
        super().__init__(master, **kw)
        self.label = tk.Label(self, text=text)
        self.label.config(width=w1)
        
        if map is None:
            map = {k:k for k in values}
        else:
            if not all(k in map for k in values):
                raise ValueError("Unmapped keys: " % " ".join(set(values)-set(map)))
        
        self.map = map
        self.events = events
        self.event = event
        
        self.menu = Combobox(self, values=values)
        self.menu.bind("<<ComboboxSelected>>", self.menu_selected)
        self.menu.config(width=w2)
        
        self.label.grid(row=0, column=0, sticky=(tk.E, tk.W, tk.N, tk.S))
        self.menu.grid(row=0, column=1, sticky=(tk.E, tk.W, tk.N, tk.S))
        
        
    def get_raw(self):
        return self.menu.get()
    
    def get(self):
        return self.map[self.menu.get()]
        
    def menu_selected(self, e):
        self.events.fire(self.event, self.get())
        
    

In [353]:
class StatusDisplay(LabelDisplayFrame):
    def __init__(self, master, text, events, cfg):
        super().__init__(master, text, events)

        # text config....
        c,n,o = cfg.simops.initial_request_cno
        a = 1-(c+n+o)
        sub2 = "\u2082:"
        co2 = "CO"+sub2
        n2 = "N"+sub2
        o2 = "O"+sub2
        
        self.add_label("PV", "PV_PROCESS_UPDATE", "PV:", cfg.simops.initial_pv, 3, 6)
        self.add_label("CO2", "CO2_PROCESS_UPDATE", co2, "%.1f%%" % (c*100), 3, 6)
        self.add_label("N2", "N2_PROCESS_UPDATE", n2, "%.1f%%" % (n*100), 3, 6)
        self.add_label("O2", "O2_PROCESS_UPDATE", o2, "%.1f%%" % (o*100), 3, 6)
        self.add_label("Air", "AIR_PROCESS_UPDATE", "Air:", "%.1f%%" % (a*100), 3, 6)
        
        self.add_label("c", "C_PROCESS_UPDATE", "c:", cfg.simops.c, 3, 6)
        self.add_label("dc", "DC_PROCESS_UPDATE", "dc:", cfg.simops.dc, 3, 6)
        self.add_label("d2c", "D2C_PROCESS_UPDATE", "d\u00B2c:", cfg.simops.d2c, 3, 6)
        
        self.events.register("PROCESS_STATE_UPDATE", self.on_state_update)
        
    def on_state_update(self, ev, state, *args):
        self.labels['c'].set("%.4g" % state['c'])
        self.labels['dc'].set("%.4g" % state['dc'])
        self.labels['d2c'].set("%.4g" % state['d2c'])
        
        self.labels['PV'].set("%.2f%%" % state['pv'])
        self.labels["CO2"].set("%.1f%%" % (state['co2_req']*100))
        self.labels["N2"].set("%.1f%%" % (state['n2_req']*100))
        self.labels["O2"].set("%.1f%%" % (state['o2_req']*100))
        self.labels["Air"].set("%.1f%%" % (state['air_req']*100))
        
class PIDFrame(LabelFrameWithLabelEntries):
    def __init__(self, master, text, events, prefix, pid_ops):
        super().__init__(master, text, events)
        
        self.add_entry("PGain:", pid_ops.p, prefix + "_PGAIN_CHANGED")
        self.add_entry("ITime:", pid_ops.i, prefix + "_ITIME_CHANGED")
        self.add_entry("DTime:", pid_ops.d, prefix + "_DTIME_CHANGED")
        self.add_entry("Beta:", pid_ops.beta, prefix + "_BETA_CHANGED")
        
        # Modes
        modes = {
            "Auto": 0,
            "Man": 1,
            "Off": 2
        }
        mode_order = list(modes)
        self.menu = MenuLabel(self, "Mode:", events, prefix + "_MODE_CHANGED", mode_order, map=modes)
        self.menu.grid(row=len(self.entries), column=0, sticky=(tk.E,tk.W,tk.N,tk.S))
        

In [354]:

class PIDSimFrame():
    def __init__(self, master, WindowClass, sim_cfg, events=None):
        if events is None:
            events = EventDispatcher()
        self.cfg = sim_cfg
        ops = cfg.simops
        self.events = events
        self.frame = tk.Frame(master)
        self.w = WindowClass(self.frame, sim_cfg, events)
        self.w.pack(side=tk.LEFT, fill=tk.BOTH, expand=1)
        self.settings_frame = sf = tk.Frame(self.frame)
        self.settings_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=1)
        
        # Widgets
        
        # N2 PID
        n2pid = PIDFrame(sf, "N\u2082 PID:", events, "N2", ops.n2_pid)

        # O2 PID
        o2pid = PIDFrame(sf, "O\u2082 PID:", events, "O2", ops.o2_pid)  
        
        # Process values
        proc = LabelFrameWithLabelEntries(sf, "Process:", events)
        proc.add_entry("Set Point:", ops.set_point, "SP_CHANGED")
        proc.add_entry("PV:", ops.initial_pv, "PV_CHANGED")
        proc.add_entry("k:", ops.k, "K_CHANGED")
        proc.add_entry("k_mult:", ops.k_mult, "KMULT_CHANGED")
        
        # Simulation options
        sim = LabelFrameWithLabelEntries(sf, "Simulation:", events)
        sim.add_entry("time factor:", cfg.time_factor, "TIMEFACTOR_CHANGED")
        sim.add_entry("x window(hr):", cfg.xwindow_hrs, "XWINDOW_CHANGED")
        
        # buttons
        clear = ttk.Button(sf, text="Clear", command=events.create_callback("CLEAR_ALL_DATA"))
        advance = EntryButton(sf, "Advance(s)", events, "ADVANCE_SIM")

        # Consumption constants
        c_frame = LabelFrameWithLabelEntries(sf, "Consumption:", events)
        c_frame.add_entry("c", ops.c, "C_CHANGED")
        c_frame.add_entry("dc/dt", 0, "DC_CHANGED")
        c_frame.add_entry("d\u00B2c/dt\u00B2", 0, "D2C_CHANGED")
        c_frame.set_trigger_mode("always")
        
        # Output display
        cstate = StatusDisplay(sf, "Current:", events, self.cfg)    
        
        tk_all = (tk.E, tk.W, tk.N, tk.S)
        n2pid.grid(row=1, column=1, sticky=tk_all)
        o2pid.grid(row=1, column=2, sticky=tk_all)
        
        proc.grid(row=2, column=1, sticky=tk_all)
        sim.grid(row=2, column=2, sticky=tk_all)

        c_frame.grid(row=3, column=1, rowspan=3, sticky=(tk.E, tk.W, tk.N, tk.S))
        cstate.grid(row=3, column=2, rowspan=8, sticky=(tk.E, tk.W, tk.N, tk.S))
        
        advance.grid(row=11, column=1)
        clear.grid(row=11, column=2, sticky=tk.S)
        
        self.wl = {}
        self.wl.update((k,v) for k,v in locals().items() if isinstance(v, tk.Widget))
        
        self.w.start()
        master.register_destroy_callback(self.w.stop)
        
        self.n2pid = n2pid
        self.o2pid = o2pid
        
        
    def pack(self, **kw):
        self.frame.pack(**kw)
        
        

In [355]:

cfg.simops.o2_pid.p = 10
cfg.simops.o2_pid.i = 80
cfg.simops.o2_pid.amax = 100
cfg.simops.o2_pid.deadband = 0
cfg.simops.o2_pid.beta = 0
cfg.simops.o2_pid.man_request = 10

cfg.simops.n2_pid.p = -2
cfg.simops.n2_pid.i = 60
cfg.simops.n2_pid.amax = 90
cfg.simops.n2_pid.deadband = 0
cfg.simops.n2_pid.beta = 0
cfg.simops.n2_pid.man_request = 10

cfg.update_interval = 50
cfg.time_factor = 200

cfg.simops.initial_pv = 20
cfg.simops.set_point = 20

cfg.xwindow_hrs = 10

root = TkQuitHack()
root.wm_title("DO Simulation")
#root.geometry("%dx%d"%(600, 800))
f = PIDSimFrame(root, DOSimWindow, cfg)
f.pack()
root.mainloop()

Value Changed: sp 80.0


In [323]:
#f.w._current_state['o2_pid'].b

In [248]:
root.mainloop()

In [178]:
root = TkQuitHack()
f = MenuLabel(root, "Foo", lambda: print("Bob"), *["Foo", "Bar", "Baz"])
f.grid()
root.mainloop()

In [304]:
root.destroy()